In [1]:
import pandas as pd
import numpy as np
import copy
import re
from keras.preprocessing.text import text_to_word_sequence
from nltk import WordNetLemmatizer

/scratch/user/achadha7/.conda/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file_name = './dataset/obfuscations.xlsx'
dfs = pd.read_excel(file_name, sheet_name='Toxic triggers obfuscations').fillna("")
dfs.head(7)

,idiots,awful,stupid,hell,abortion,worst,moron,screw,terrible,suck,...,sexist,supremacists,assholes,ignorant,morons,nationalists,rubbish,ass,uneducated,idiot
0,"i,,,diioots",ÄWfu\\l,5tup1D,HE|l,aabórt~~~ioN,wo.RSt,M0RoÑ,sc~~~rrÉww,"t,,Ë®rr:blle",$u¢k,...,5éXÎsst,SÙþRéMaac¡57ss,Áss5HOl??ËS,ì6Nõrån-t,M0®o¬¬¬n;;;s,nattioonâLi*s+5,Rú3BiSH,ãs**S,uñe\dµca+eD,iDÌôT
1,ìDi¿¿oo+5,ÄWffùL,"s7Úp""""""i|D",hæLll,áb~oRt;;;i**o??n,wÒrstt,"m,,0RÕN",sc..rrëw,têr:::RIbllê,Succk,...,$éXìstt,SuPremá{ïst$,Às5hôllÆ5,î6n_Ôr__Än~t,MÒrÒñS,Ñat^^|ooNa--lli==st==5,Ruþb?i\\s¬¬h,asss,unnÉdÜCÁtted,ÏdiÒ+
2,ÌDió7s,ªwFû¡,s::Tup!dd,hÊlll,ãbort¬¬íÖN,wÕrs==+,mmor|||0n,Sccr¨¨¨ÉW,TeRrÏb··llÈ,5u<k,...,$ê*¡5+,5ùP®e¿¿¿m~~áCistS,á5shhoolè$,igNO®Ân..tt,MoR0ns,ña+i:ooña!i...s+S,"r""""úbßi~~5#",as\5,"u//Ne;;;du,,<ate||d",îDiÔt
3,ïD¡o\\\+S,aawwfù!,s7ÚpÍdd,hël|,"Æbört¿¿¿i""""""öN",Wo^^^rr5+,"m???o""rooñ",s{Rew,Te:rRiþLè,sµc::k,...,$e%i???st,$up;;®ém__a©Ìss7s,a*ss#ool::è5,iggn0Råñt,m ör--Óñ5,n¨¨at-!ONâ£i¨¨¨s+5,ruBþi?5h,å5s,une¨du¬CaatÉD,i\di0tt
4,i¨¨¨d¨¨íOtss,awFuu£,st__üpÏdd,hel?!,aßÕRTÏoon,"w""""""or\s¬+",MóròÑ,scr3w,7e===rrr1bbllæ,ssÜcK,...,s¬¬ÈX!ssT,supremácìsts,a55holÉS,!GNo¨¨r___añ+,µo--Roñ$,"natïÖñÄ¡i sst""""""S",rrü83ish,àS5,Vne??Ducättëd,ÎDÌo..t
5,Ìdi'''ôTs,awfu!,stuupID,#ë¡L,âþo?Rtion,wO®s...T,MÔroon,sc..rre;w,te_rRibl~~Ë,5Vþk,...,$e¬¬xIsT,Su..p//r¿¿¿ema·cÎ5tts,â$ShO¡eeS,:6ÑºrÄñ7,"µo~~r''o'''n""""""5",nna^^^+i¿¿¿o;;;näll|s\T5,r???Vb???bbi¬SH,as_5,ûNed**uca===+Éd,idiiOT
6,Id^^:ôt$,"a_w,,Fü/",sstvpiD,He__lL,ÂböRTîon,wwÒr?s::T,"m,o|||ron",ss¢RÉw,+È®RÎb¡ee,s··uucK,...,5e*i***st,"5ÙþrÆMa""""Çí$+5",Á5SHoolee5,i···GNOraÑt,m**oRóÑs,ÑaattÎoNa---¡iis¬¬T$,r'Ùb6îsH,"ås""""""s",UnÉdÙ{AtÉD,ÎDi·o//+


In [3]:
class BaseTokenizer(object):
    def process_text(self, text):
        raise NotImplemented

    def process(self, texts):
        for text in texts:
            yield self.process_text(text)

In [4]:
RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ] + [re.escape(a) for a in list(dfs["ass"].values)],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' trans gender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k'
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit'
        ],

    ' homo sex ual':
        [
            'homosexual'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                      'i d i o t'
        ] + [re.escape(a) for a  in list(dfs["idiot"].values)],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],

    ' shit hole ':
        [
            'shythole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
}

In [5]:
RE_PATTERNS['idiots'] = [re.escape(a) for a in list(dfs["idiots"].values)]
RE_PATTERNS['awful']  =  [re.escape(a) for a in list(dfs["awful"].values)]
RE_PATTERNS['stupid']  =  [re.escape(a) for a in list(dfs["stupid"].values)]
RE_PATTERNS['hell']  =  [re.escape(a) for a in list(dfs["hell"].values)]
RE_PATTERNS['abortion']  =  [re.escape(a) for a in list(dfs["abortion"].values)]
RE_PATTERNS['worst']  =  [re.escape(a) for a in list(dfs["worst"].values)]
RE_PATTERNS['moron']  =  [re.escape(a) for a in list(dfs["moron"].values)]
RE_PATTERNS['screw']  =  [re.escape(a) for a in list(dfs["screw"].values)]
RE_PATTERNS['terrible']  =  [re.escape(a) for a in list(dfs["terrible"].values)]
RE_PATTERNS['suck']  =  [re.escape(a) for a in list(dfs["suck"].values)]
RE_PATTERNS['shame']  =  [re.escape(a) for a in list(dfs["shame"].values)]
RE_PATTERNS['racist']  =  [re.escape(a) for a in list(dfs["racist "].values)]
RE_PATTERNS['supremacists']  =  [re.escape(a) for a in list(dfs["supremacists"].values)]
RE_PATTERNS['assholes']  =  [re.escape(a) for a in list(dfs["assholes"].values)]
RE_PATTERNS['ignorant']  =  [re.escape(a) for a in list(dfs["ignorant"].values)]
RE_PATTERNS['morons']  =  [re.escape(a) for a in list(dfs["morons"].values)]
RE_PATTERNS['nationalists']  =  [re.escape(a) for a in list(dfs["nationalists"].values)]
RE_PATTERNS['rubbish']  =  [re.escape(a) for a in list(dfs["rubbish"].values)]
# RE_PATTERNS['ass']  =  [re.escape(a) for a in list(dfs["ass"].values)]
RE_PATTERNS['uneducated']  = [re.escape(a) for a in list(dfs["uneducated"].values)]
#RE_PATTERNS['idiot']  =  [re.escape(a) for a in list(dfs["idiot"].values)]

In [21]:
RE_PATTERNS['idiots'] = list(dfs["idiots"].values)
RE_PATTERNS['awful']  = list(dfs["awful"].values)
RE_PATTERNS['stupid']  = list(dfs["stupid"].values)
RE_PATTERNS['hell']  = list(dfs["hell"].values)
RE_PATTERNS['abortion']  = list(dfs["abortion"].values)
RE_PATTERNS['worst']  = list(dfs["worst"].values)
RE_PATTERNS['moron']  = list(dfs["moron"].values)
RE_PATTERNS['screw']  = list(dfs["screw"].values)
RE_PATTERNS['terrible']  = list(dfs["terrible"].values)
RE_PATTERNS['suck']  = list(dfs["suck"].values)
RE_PATTERNS['shame']  = list(dfs["shame"].values)
RE_PATTERNS['racist']  = list(dfs["racist "].values)
RE_PATTERNS['supremacists']  = list(dfs["supremacists"].values)
RE_PATTERNS['assholes']  = list(dfs["assholes"].values)
RE_PATTERNS['ignorant']  = list(dfs["ignorant"].values)
RE_PATTERNS['morons']  = list(dfs["morons"].values)
RE_PATTERNS['nationalists']  = list(dfs["nationalists"].values)
RE_PATTERNS['rubbish']  = list(dfs["rubbish"].values)
# RE_PATTERNS['ass']  = list(dfs["ass"].values)
RE_PATTERNS['uneducated']  = [re.escape(a) for a in list(dfs["uneducated"].values)]
#RE_PATTERNS['idiot']  = list(dfs["idiot"].values)

In [6]:
class PatternTokenizer(BaseTokenizer):
    def __init__(self, lower=True, initial_filters=r"[^a-z0-9!@#\$%\^\&\*_\-,\.' ]", patterns=RE_PATTERNS,
                 remove_repetitions=True):
        self.lower = lower
        self.patterns = patterns
        self.initial_filters = initial_filters
        self.remove_repetitions = remove_repetitions

    def process_text(self, text):
        x = self._preprocess(text)
        for target, patterns in self.patterns.items():
            for pat in patterns:
                x = re.sub(pat, target, x)
        x = re.sub(r"[^a-z' ]", ' ', x)
        return x.split()

    def process_ds(self, ds):
        ### ds = Data series

        # lower
        ds = copy.deepcopy(ds)
        if self.lower:
            ds = ds.str.lower()
        # remove special chars
        if self.initial_filters is not None:
            ds = ds.str.replace(self.initial_filters, ' ')
        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
            ds = ds.str.replace(pattern, r"\1")

        for target, patterns in self.patterns.items():
            for pat in patterns:
                ds = ds.str.replace(pat, target)

        ds = ds.str.replace(r"[^a-z' ]", ' ')

        return ds.str.split()

    def _preprocess(self, text):
        # lower
        if self.lower:
            text = text.lower()

        # remove special chars
        if self.initial_filters is not None:
            text = re.sub(self.initial_filters, ' ', text)

        # fuuuuck => fuck
        if self.remove_repetitions:
            pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
            text = pattern.sub(r"\1", text)
        return text
        


In [7]:
def clean_comment(comment):
   
    tokenizer = PatternTokenizer()
    d = {'comment_text': [comment]}
    df = pd.DataFrame(data=d)
    return tokenizer.process_ds(df["comment_text"]).str.join(sep=" ")[0]

In [ ]:
#print(clean_comment("naziii asshole cunt face bittchh go fuuuck uaself u trannie hoe3 tra@mp poopieie basstarrrd f##k")[0])
print(clean_comment("nazi")[0])